<a href="https://colab.research.google.com/github/PillB/CoderhouseDL3/blob/main/ProjectoFinal%2BIllescas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import requests

# Descargar recursos necesarios de nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Descargar el texto de "Pride and Prejudice" desde Project Gutenberg
# URL del texto completo en formato .txt
url = 'https://www.gutenberg.org/files/1342/1342-0.txt'
response = requests.get(url)
text_data = response.text

# Convertir el texto en una lista de líneas para formar un dataframe
lines = text_data.split('\n')
df = pd.DataFrame(lines, columns=['text'])

# Mostrar las primeras filas del dataset para validar la carga
print("Primeras filas del dataset:")
print(df.head())

# 1. Tokenización
# Explicación: La tokenización separa el texto en palabras individuales o 'tokens'.
df['tokens'] = df['text'].apply(lambda x: word_tokenize(x.lower()))

# 2. Eliminación de Stopwords
# Explicación: Las stopwords son palabras comunes que no aportan mucho valor analítico (e.g., 'el', 'la', 'y').
stop_words = set(stopwords.words('english'))  # Cambiar a inglés porque el texto es en inglés
df['tokens_sin_stopwords'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

# 3. Lematización
# Explicación: La lematización reduce las palabras a su forma base o lema.
lemmatizer = WordNetLemmatizer()
df['lemmas'] = df['tokens_sin_stopwords'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# 4. Análisis de Sentimientos
# Explicación: Analizamos la polaridad de cada oración usando TextBlob.
df['sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)

# 5. Vectorización TF-IDF
# Explicación: El TF-IDF evalúa la importancia de palabras en el texto basado en frecuencia.
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['text'])

# Mostrar algunos resultados del preprocesamiento
print("Resultados del preprocesamiento:")
print(df.head())

# Verificar que se cumplieron todos los requerimientos
# - Tokenización: Sí
# - Eliminación de Stopwords: Sí
# - Lematización: Sí
# - Análisis de Sentimientos: Sí
# - TF-IDF Vectorización: Sí


In [ ]:
# Importar librerías necesarias para el modelado de redes neuronales
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np

# Cargar y preparar el dataset MNIST
# Este conjunto de datos contiene imágenes de dígitos escritos a mano de 0 a 9
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalizar los valores de píxeles entre 0 y 1

# Mostrar las dimensiones de los conjuntos de datos
print(f"Dimensiones del conjunto de entrenamiento: {x_train.shape}")
print(f"Dimensiones del conjunto de prueba: {x_test.shape}")

# Mostrar la distribución de etiquetas en el conjunto de entrenamiento
unique, counts = np.unique(y_train, return_counts=True)
print(f"Distribución de etiquetas en el conjunto de entrenamiento: {dict(zip(unique, counts))}")

# Visualización exploratoria de algunas imágenes
plt.figure(figsize=(10, 5))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    plt.imshow(x_train[i], cmap='gray')
    plt.title(f"Etiqueta: {y_train[i]}")
    plt.axis('off')
plt.show()

# Construir y compilar un modelo de red neuronal simple con dos capas
model = Sequential([
    Flatten(input_shape=(28, 28)),        # Aplanar las imágenes de 28x28 píxeles en vectores 1D
    Dense(128, activation='relu'),        # Primera capa densa con 128 neuronas y activación ReLU
    Dense(10, activation='softmax')       # Capa de salida con 10 neuronas para la clasificación (0-9)
])

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo en los datos de entrenamiento
history = model.fit(x_train, y_train, epochs=5, validation_split=0.2)

# Evaluar el modelo en los datos de prueba
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Precisión en los datos de prueba: {test_acc:.4f}")

# Graficar la precisión del entrenamiento y la validación
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Precisión del Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión de la Validación')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.title('Precisión de Entrenamiento y Validación')
plt.legend()
plt.show()

# Análisis Detallado de Resultados:
# - Las precisiones de entrenamiento y validación muestran el proceso de aprendizaje.
# - La precisión en el conjunto de prueba proporciona una medida de la capacidad de generalización del modelo.

# Conclusiones
# - El modelo actual proporciona una línea base con precisión aceptable.
# - Las mejoras futuras pueden incluir agregar capas convolucionales o ajustar hiperparámetros.
